In [ ]:
import numpy as np
import pyscf
from tn4qa.dmrg import DMRG
from tn4qa.qi_metrics import get_all_mutual_information
from tn4qa.utils import ham_dict_from_scf
from tn4qa.mps import MatrixProductState
import json
from qiskit import QuantumCircuit

Rs = np.linspace(0.8, 3.0, 23)

results = {}
fci = np.array([-106.76630792, -107.29307577, -107.54930096, -107.65412245,
       -107.67733975, -107.65936967, -107.62317418, -107.58163492,
       -107.54208577, -107.50872873, -107.48345735, -107.46611285,
       -107.4551556 , -107.44863242, -107.44485855, -107.44265552,
       -107.44130628, -107.4404165 , -107.43978533, -107.43930775,
       -107.43895616, -107.43858732, -107.43844536])
FCI_energies = {Rs[idx]:fci[idx] for idx in range(len(fci))}

qc = QuantumCircuit(20)
for i in range(14):
    qc.x(i)
input_mps = MatrixProductState.from_qiskit_circuit(qc, 16)
input_mps = input_mps.expand_bond_dimension_list(1, list(range(1,20)))

for R in Rs:
    print(R)
    fci_energy = FCI_energies[R]
    R_str = str(round(R, 1))
    results[R_str] = {}

    mol = pyscf.M(
        atom='N 0 0 0; N 0 0 {}'.format(R),
        basis='STO-3G',
    )
    mf = pyscf.scf.RHF(mol).run(verbose=0)
    
    norb = mol.nao
    nocc = np.count_nonzero(mf.mo_occ)
    nvir = mol.nao - np.count_nonzero(mf.mo_occ)

    mycc = pyscf.cc.CCSD(mf).run(verbose=0).e_tot
    # myfci = pyscf.fci.FCI(mf).run(verbose=0)

    results[R_str]['norb'] = norb
    results[R_str]['nocc'] = nocc
    results[R_str]['nvir'] = nvir
    
    results[R_str]['E_RHF'] = mf.e_tot
    # results[R_str]['E_CCSD'] = mycc.e_tot
    # results[R_str]['E_FCI'] = myfci.e_tot

    results[R_str]['mo_energy_RHF'] = mf.mo_energy.tolist()

    ham = ham_dict_from_scf(mf, qubit_transformation="JW")
    dmrg = DMRG(
        hamiltonian=ham,
        max_mps_bond=32,
        method="one-site",
        initial_state=input_mps
    )
    if R == 0.8:
        runs = 10
    else:
        runs = 4
    dmrg.run(runs)
    print(mycc - fci_energy, dmrg.energy - fci_energy)
    input_mps = dmrg.mps
    results[R_str]['E_DMRG'] = dmrg.energy
    
    # mi_matrix = get_all_mutual_information(dmrg.mps)

    # results[R_str]['mi_matrix'] = mi_matrix.tolist()

# with open('results.json', 'w') as f:
#     json.dump(results, f, indent=4)


0.8
0.00113316589224155 0.02779970821067934
0.9
0.001701733190245136 0.009875130142901867
1.0
0.0025780364261862587 0.008874972571035755
1.1
0.003924985773394951 0.007396960738276448
1.2000000000000002
0.00590065278358054 0.0029577320818958697
1.3
0.008541589382474513 0.00030295215154296784
1.4000000000000001
